<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/gemma2_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers bitsandbytes accelerate beautifulsoup4

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import os
import torch
import requests

from bs4 import BeautifulSoup
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


os.environ["HUGGING_FACE_HUB_TOKEN"] = userdata.get('HF_TOKEN')
GEMMA = "google/gemma-2-2b-it"

In [ ]:
!nvidia-smi

Thu Aug  1 17:40:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Standard approach

In [ ]:
def get_tok_model(name):
  tokenizer = AutoTokenizer.from_pretrained(name)
  model = AutoModelForCausalLM.from_pretrained(name, device_map="auto", torch_dtype=torch.bfloat16)
  return tokenizer, model

In [ ]:
tokenizer, model = get_tok_model(GEMMA)

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [{"role": "user", "content": "Briefly explain the concept of flash attention and its difference from original attention mechanism."}]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Briefly explain the concept of flash attention and its difference from original attention mechanism.<end_of_turn>
* **Flash Attention** is a novel attention mechanism that aims to improve the efficiency of attention computation by reducing the number of computations required. It achieves this by using a "flash" of attention computation, where the attention weights are calculated for a small subset of the input sequence. 

**Original Attention Mechanism** (e.g., self-attention in Transformer) calculates attention weights for all possible pairs of tokens in the sequence. This can be computationally expensive, especially for long sequences.

**Key Differences:**

* **Computational Efficiency:** Flash attention significantly reduces the number of computations compared to original attention mechanisms.
* **Memory Efficiency:** Flash attention requires less memory to store attention weights, as it only needs to store the attention weights for the subset of tokens.
* 

In [ ]:
!nvidia-smi

Thu Aug  1 17:23:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              30W /  70W |   5405MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
def print_gpu_memory_usage():
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024 ** 3:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved() / 1024 ** 3:.2f} GB")


def inference_gemma(content, model, tokenizer, max_new_tokens=256):

  # Prepare the input
  messages = [{"role": "user", "content": content}]
  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

  # Check GPU memory usage after preparing input
  print("\nGPU memory usage after preparing input:")
  print_gpu_memory_usage()

  # Generate response
  outputs = model.generate(**input_ids, max_new_tokens=max_new_tokens)

  # Check GPU memory usage after generation
  print("\nGPU memory usage after generation:")
  print_gpu_memory_usage()

  # Decode and print the output
  results = tokenizer.decode(outputs[0])
  print(results)

In [ ]:
%%time
inference_gemma("What is the main idea behind flash attention?",
                model,
                tokenizer)

Initial GPU memory usage:
Allocated: 4.91 GB
Reserved: 5.15 GB

GPU memory usage after preparing input:
Allocated: 4.91 GB
Reserved: 5.15 GB

GPU memory usage after generation:
Allocated: 4.91 GB
Reserved: 5.15 GB
<bos><start_of_turn>user
What is the main idea behind flash attention?<end_of_turn>
* **Attention is a mechanism that allows a model to focus on specific parts of the input sequence.**
* **Flash attention is a technique that speeds up attention computation by using a "flash" of attention.**
* **Flash attention is a type of attention mechanism that uses a single attention head to process the entire input sequence.**
* **Flash attention is a technique that reduces the number of attention heads used in a model.**

**The correct answer is:**

**Flash attention is a technique that speeds up attention computation by using a "flash" of attention.**

**Explanation:**

Flash attention is a specific approach to attention mechanisms that aims to improve efficiency. It achieves this by f

## Quantized

In [4]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [5]:
def get_quant(name, quantization_config):
  tokenizer = AutoTokenizer.from_pretrained(name)
  model = AutoModelForCausalLM.from_pretrained(name, quantization_config=quantization_config)
  return tokenizer, model

In [ ]:
tokenizer, model= get_quant(GEMMA, quantization_config)

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

In [8]:
!nvidia-smi

Thu Aug  1 18:30:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0              27W /  70W |   2427MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
input_text = "Explain flash attention mechanism to me as if I were 12 years old."
inference_gemma(input_text, model, tokenizer, 50)


GPU memory usage after preparing input:
Allocated: 2.22 GB
Reserved: 2.25 GB


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(



GPU memory usage after generation:
Allocated: 2.25 GB
Reserved: 2.43 GB
<bos><start_of_turn>user
Explain flash attention mechanism to me as if I were 12 years old.<end_of_turn>


Imagine you're reading a long story. You don't just read every word at once, right? You focus on important parts, like the main characters, the plot twists, and the exciting action scenes. 

That's what a flash attention mechanism does! It helps computers understand and process information by focusing on the most important parts of a big data set. 

Think of it like a spotlight on a stage. The spotlight shines on the most important parts of the story, making them stand out and easier to understand. 

The spotlight helps the computer understand the story better by:

* **Identifying the most important parts:** It helps the computer focus on the most important information in a big data set.
* **Remembering the important parts:** It helps the computer remember the important information it has seen.
* **Making con

In [ ]:
%%time
input_text = "Explain flash attention mechanism to me as if I were 12 years old and how is it different from usual attention mechanism."
inference_gemma(input_text, model, tokenizer, 128+64)


GPU memory usage after preparing input:
Allocated: 2.25 GB
Reserved: 2.43 GB

GPU memory usage after generation:
Allocated: 2.25 GB
Reserved: 2.43 GB
<bos><start_of_turn>user
Explain flash attention mechanism to me as if I were 12 years old and how is it different from usual attention mechanism.<end_of_turn>
*Imagine you're trying to read a long story. You don't read every word at once, right? You focus on the most important parts and skip over the boring parts.*

**That's kind of like how attention works in computers.**

**Usual attention:** Imagine you have a big box of LEGOs. You want to build a spaceship. You look at each LEGO brick individually, and decide which ones are important for the spaceship. This is like the usual attention mechanism. It's like looking at each word in a sentence individually.

**Flash attention:** Now imagine you have a big box of LEGOs, but you have a special tool that lets you quickly scan the whole box and see which bricks are important for building th

In [16]:
import gc
del model
del tokenizer
gc.collect()

279

In [ ]:
def model_classifier(content, model, tokenizer):
  messages = [
        {
            "role": "user",
            "content": (
                f"You are an AI tasked with analyzing HTML content to determine if it is a phishing page. "
                f"Carefully scan the provided HTML content and determine whether it is phishing or legitimate. "
                f"Your response should be in the format:\n"
                f"RESULT: [phishing or legitimate]\n"
                f"Explanation: [brief explanation why you reached this conclusion]\n"
                f"Below is the content:\n{content}"
                        )},]

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

  outputs = model.generate(**input_ids, max_new_tokens=256)
  print(tokenizer.decode(outputs[0]))

In [ ]:
def fetch_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return None

def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup


def extract_main_content(soup):
    # Try to find the main content using common tags and classes
    main_content = None

    main_tags = ['main', 'article']
    main_classes = ['main-content', 'content', 'main', 'article']

    for tag in main_tags:
        main_content = soup.find(tag)
        if main_content:
            break

    if not main_content:
        for class_name in main_classes:
            main_content = soup.find(class_=class_name)
            if main_content:
                break

    # If main content is not found, fall back to the body tag
    if not main_content:
        main_content = soup.find('body')

    return main_content

In [ ]:
html_content = fetch_html('https://accounts.google.com')
if html_content:
  soup = parse_html(html_content)
  main_content = extract_main_content(soup)
  print(soup.prettify())

The model throws error CUDA out of memory if the input size is too long

In [ ]:
text = """<!DOCTYPE html>
<html dir="ltr" lang="en-US">
 <head>
  <base href="https://accounts.google.com/v3/signin/"/>
  <link href="//www.gstatic.com" rel="preconnect"/>
  <meta content="origin" name="referrer"/>
  <style data-href="https://www.gstatic.com/_/mss/boq-identity/_/ss/k=boq-identity.AccountsSignInUi.WfpSiGW1k78.L.W.O/am=hD4Yg9FtBIICIAIVACAAUAAAAAAAAAAAAAwAACIh/d=1/ed=1/rs=AOaEmlH0kj4DhxdPSklXBwlyRq5nvBmaPQ/m=identifierview,_b,_tp" nonce="TaSeLqc4uqf3ONIHP6MfHw">
   c-wiz{contain:style}c-wiz>c-data{display:none}c-wiz.rETSD{contain:none}c-wiz.Ubi8Z{contain:layout style}@-webkit-keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2);-webkit-transform:translate(-50%,-50%) scale(0.2);transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2);-webkit-transform:translate(-50%,-50%) scale(2.2);transform:translate(-50%,-50%) scale(2.2)}}@keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2);-webkit-transform:translate(-50%,-50%) scale(0.2);transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2);-webkit-transform:translate(-50%,-50%) scale(2.2);transform:translate(-50%,-50%) scale(2.2)}}@-webkit-keyframes quantumWizIconFocusPulse{0%{-webkit-transform:translate(-50%,-50%) scale(1.5);-webkit-transform:translate(-50%,-50%) scale(1.5);transform:translate(-50%,-50%) scale(1.5);opacity:0}to{-webkit-transform:translate(-50%,-50%) scale(2);-webkit-transform:translate(-50%,-50%) scale(2);transform:translate(-50%,-50%) scale(2);opacity:1}}@keyframes quantumWizIconFocusPulse{0%{-webkit-transform:translate(-50%,-50%) scale(1.5);-webkit-transform:translate(-50%,-50%) scale(1.5);transform:translate(-50%,-50%) scale(1.5);opacity:0}to{-webkit-transform:translate(-50%,-50%) scale(2);-webkit-transform:translate(-50%,-50%) scale(2);transform:translate(-50%,-50%) scale(2);opacity:1}}@-webkit-keyframes quantumWizRadialInkSpread{0%{-webkit-transform:scale(1.5);-webkit-transform:scale(1.5);transform:scale(1.5);opacity:0}to{-webkit-transform:scale(2.5);-webkit-transform:scale(2.5);transform:scale(2.5);opacity:1}}@keyframes quantumWizRadialInkSpread{0%{-webkit-transform:scale(1.5);-webkit-transform:scale(1.5);transform:scale(1.5);opacity:0}to{-webkit-transform:scale(2.5);-webkit-transform:scale(2.5);transform:scale(2.5);opacity:1}}@-webkit-keyframes quantumWizRadialInkFocusPulse{0%{-webkit-transform:scale(2);-webkit-transform:scale(2);transform:scale(2);opacity:0}to{-webkit-transform:scale(2.5);-webkit-transform:scale(2.5);transform:scale(2.5);opacity:1}}@keyframes quantumWizRadialInkFocusPulse{0%{-webkit-transform:scale(2);-webkit-transform:scale(2);transform:scale(2);opacity:0}to{-webkit-transform:scale(2.5);-webkit-transform:scale(2.5);transform:scale(2.5);opacity:1}}.JnOM6e{background-color:transparent;border:none;border-radius:4px;box-sizing:border-box;display:inline-block;font-size:14px;height:36px;letter-spacing:.15px;line-height:34px;padding:0 24px;position:relative;text-align:center}.JnOM6e:focus-visible{outline:none;position:relative}.JnOM6e:focus-visible::after{border:2px solid rgb(24,90,188);border-radius:6px;bottom:-4px;box-shadow:0 0 0 2px rgb(232,240,254);content:"";left:-4px;position:absolute;right:-4px;top:-4px}.rDisVe:focus:not(:focus-visible),.GjFdVe:focus:not(:focus-visible){box-shadow:0 1px 1px 0 rgba(66,133,244,.3),0 1px 3px 1px rgba(66,133,244,.15)}.rDisVe:hover:not(:focus-visible),.GjFdVe:hover:not(:focus-visible){box-shadow:0 1px 1px 0 rgba(66,133,244,.45),0 1px 3px 1px rgba(66,133,244,.3)}.JnOM6e:disabled{pointer-events:none}.JnOM6e:hover{cursor:pointer}.JnOM6e.kTeh9{line-height:36px;text-decoration:none}.JnOM6e.eLNT1d{display:none}.rDisVe{background-color:rgb(26,115,232);color:#fff}.rDisVe:disabled{background-color:rgb(232,234,237);color:rgb(154,160,166)}.rDisVe:focus,.rDisVe:hover{background-color:rgb(24,90,188)}.GjFdVe{border:1px solid rgb(218,220,224);color:rgb(26,115,232)}.GjFdVe:disabled{color:rgb(189,193,198)}.GjFdVe:active{background-color:rgb(174,203,250);color:rgb(23,78,166)}.GjFdVe:focus{background-color:rgb(232,240,254);border-color:rgb(24,90,188);color:rgb(23,78,166)}.GjFdVe:hover{background-color:rgb(232,240,254);color:rgb(23,78,166)}.KXbQ4b{color:rgb(26,115,232);min-width:0;padding-left:8px;padding-right:8px}.KXbQ4b:disabled{color:rgb(189,193,198)}.KXbQ4b:active,.KXbQ4b:hover{color:rgb(24,90,188)}.KXbQ4b:active{background-color:rgba(26,115,232,.12)}.KXbQ4b:focus,.KXbQ4b:hover{background-color:rgba(26,115,232,.04)}.hZUije{border:0}.hZUije.WS4XDd{max-height:1.3333em;padding:0 2px;vertical-align:middle;width:auto}.kHluYc{border:0;object-fit:contain}.kHluYc.WS4XDd{max-height:1.3333em;padding:0 2px;vertical-align:middle;width:auto}.aN1Vld{margin:16px 0;outline:none}.aN1Vld+.aN1Vld{margin-top:24px}.aN1Vld:first-child{margin-top:0}.aN1Vld:last-child{margin-bottom:0}.fegW5d{border-radius:8px;padding:16px}.fegW5d>:first-child{margin-top:0}.fegW5d>:last-child{margin-bottom:0}.fegW5d .cySqRb,.fegW5d .yOnVIb{color:rgb(32,33,36)}.fegW5d.YFdWic .yOnVIb{color:rgb(95,99,104);margin-top:4px;padding:0}.fegW5d.YFdWic .wSQNd,.fegW5d.YFdWic .yOnVIb{margin-left:64px;width:calc(100% - 64px)}.fegW5d.YFdWic:not(.S7S4N) .wSQNd{margin-left:0;width:0}.fegW5d:not(.S7S4N)>.yOnVIb{margin-top:0}.fegW5d.sj692e{background:rgb(232,240,254)}.fegW5d.Xq8bDe{background:rgb(252,232,230)}.fegW5d.rNe0id{background:rgb(254,247,224)}.fegW5d.YFdWic{border:1px solid rgb(218,220,224);min-height:80px;position:relative}.fegW5d:not(.S7S4N){display:-webkit-box;display:-webkit-flex;display:flex}.aN1Vld.eLNT1d{display:none}.aN1Vld.RDPZE{opacity:.5;pointer-events:none}.aN1Vld.RDPZE .aN1Vld.RDPZE{opacity:1}.wlrzMe{border:1px solid rgb(218,220,224);border-radius:8px;padding:16px}.wlrzMe .EEiyWe{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-pack:end;-webkit-justify-content:flex-end;justify-content:flex-end;margin-top:16px}.wlrzMe .EEiyWe .GjFdVe{margin-bottom:0;margin-top:0}.wSQNd:empty{display:none}.wSQNd>:first-child{margin-top:0;padding-top:0}.wSQNd>:last-child{margin-bottom:0;padding-bottom:0}.cySqRb{-webkit-box-align:center;-webkit-align-items:center;align-items:center;color:rgb(32,33,36);display:-webkit-box;display:-webkit-flex;display:flex;font-size:16px;font-weight:500;letter-spacing:.1px;line-height:1.4286;margin-bottom:0;margin-top:0;padding:0}.zlrrr{color:rgb(95,99,104);-webkit-flex-shrink:0;flex-shrink:0;height:20px;margin-right:16px;width:20px}.zlrrr .GxLRef{height:100%;width:100%}.fegW5d .zlrrr{margin-top:0}.fegW5d.sj692e .zlrrr{color:rgb(25,103,210)}.fegW5d.Xq8bDe .zlrrr{color:rgb(197,34,31)}.fegW5d.rNe0id .zlrrr{color:rgb(234,134,0)}.fegW5d.YFdWic .zlrrr{height:48px;left:16px;position:absolute;top:16px;width:48px}.yOnVIb{margin:auto -24px;padding-left:24px;padding-right:24px;margin-bottom:16px;margin-top:10px}.wlrzMe .yOnVIb{margin-bottom:0;margin-top:16px}.yOnVIb>:first-child:not(section){margin-top:0;padding-top:0}.yOnVIb>:last-child:not(section){margin-bottom:0;padding-bottom:0}.wSQNd:empty+.yOnVIb{margin-top:0}.yOnVIb:only-child{margin-bottom:0;margin-top:0}*,*:before,*:after{box-sizing:inherit}html{box-sizing:border-box;-webkit-tap-highlight-color:transparent}body,input,textarea,select,button{color:rgb(32,33,36);font-family:arial,sans-serif}body{background:#fff;direction:ltr;display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-webkit-flex-direction:column;flex-direction:column;font-size:14px;line-height:1.4286;margin:0;min-height:100vh;padding:0;position:relative}@media (min-width:601px){body{-webkit-box-pack:center;-webkit-justify-content:center;justify-content:center}}[data-style=heading]{color:rgb(32,33,36);font-size:16px;font-weight:500;letter-spacing:.1px;line-height:1.4286}.BDEI9 h2:not(.TrZEUc){color:rgb(32,33,36);font-size:16px;font-weight:500;letter-spacing:.1px;line-height:1.4286}.BDEI9 .fegW5d a:not(.TrZEUc),.BDEI9 .fegW5d button:not(.TrZEUc){color:rgb(25,103,210)}.BDEI9 a{text-decoration:none}.BDEI9 a:not(.TrZEUc),.BDEI9 button:not(.TrZEUc){border-radius:4px;color:rgb(26,115,232);display:inline-block;font-weight:500;letter-spacing:.25px;outline:0;position:relative;z-index:1}.BDEI9 button:not(.TrZEUc){background-color:transparent;cursor:pointer;padding:0;text-align:left}.BDEI9 button:not(.TrZEUc){border:0}.BDEI9 button::-moz-focus-inner{border:0}.BDEI9 a:not(.TrZEUc):focus::after,.BDEI9 a:not(.TrZEUc):active::after,.BDEI9 button:not(.TrZEUc):focus::after,.BDEI9 button:not(.TrZEUc):active::after{background-color:rgba(26,115,232,.15);border-radius:2px;content:"";position:absolute;inset:-2px -3px;z-index:-1}.BDEI9 img:not(.TrZEUc){border:0;max-height:1.3em;vertical-align:middle}@media (min-width:601px){.BDEI9{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-webkit-flex-direction:column;flex-direction:column;min-height:100vh;position:relative}.BDEI9:before,.BDEI9:after{-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1;content:"";height:24px}.BDEI9:before{min-height:30px}.BDEI9:after{min-height:24px}.BDEI9.LZgQXe:after{min-height:63.9996px}}.gEc4r{display:-webkit-box;display:-webkit-flex;display:flex;height:24px;-webkit-box-pack:center;-webkit-justify-content:center;justify-content:center}.gEc4r.jlqJKb{height:auto;min-height:24px;padding-top:24px}.BivnM{-webkit-box-align:center;-webkit-align-items:center;align-items:center;border-bottom:1px solid #ccc;display:-webkit-box;display:-webkit-flex;display:flex;height:36px;left:0;padding:0 16px;position:absolute;right:0;top:0}.BivnM .ji6sFc{height:14px;margin-right:12px}.O3jdWc{color:rgb(95,99,104);font-size:14px;height:14px;margin-top:-2px}.Zwk8S{display:block;height:37px;width:37px}.wFNE4e{color:rgb(26,115,232)}.HUYFt{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-flex-wrap:wrap;flex-wrap:wrap;font-size:12px;-webkit-box-pack:justify;-webkit-justify-content:space-between;justify-content:space-between;padding:0 24px 14px}@media (min-width:601px){.HUYFt{padding-left:0;padding-right:0;position:absolute;width:100%}}.hXs2T .pUP0Nd{color:rgb(60,64,67)}.hXs2T{margin-left:-16px;margin-right:16px}.N158t{-webkit-appearance:none;appearance:none;background-color:transparent;background-image:url("data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMjRweCIgdmlld0JveD0iMCAwIDI0IDI0IiB3aWR0aD0iMjRweCIgZmlsbD0iIzQ1NUE2NCI+PHBhdGggZD0iTTAgMGgyNHYyNEgwVjB6IiBmaWxsPSJub25lIi8+PHBhdGggZD0iTTcgMTBsNSA1IDUtNUg3eiIvPjwvc3ZnPg==");background-position:right;background-repeat:no-repeat;border:none;border-radius:4px;color:rgb(95,99,104);cursor:pointer;font-size:12px;line-height:15.9996px;outline:none;padding:16.0002px 24px 16.0002px 16px}.N158t:focus{background-color:rgb(232,234,237)}.M2nKge{list-style:none;margin:0 -16px;padding:0}.vomtoe{display:inline-block;margin:0}.pUP0Nd{background-color:transparent;border-radius:4px;color:rgb(95,99,104);display:inline-block;line-height:15.9996px;outline:none;padding:16px}.pUP0Nd:focus{background-color:rgb(232,234,237)}.vOZun,.gomQac{padding-bottom:3px;padding-top:9px;margin-bottom:0;margin-top:0}.gomQac{margin:0}.vOZun:empty{display:none}.Ha17qf{background:#fff;display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-webkit-flex-direction:column;flex-direction:column;max-width:100%;min-height:100vh;position:relative}@media (min-width:601px){.Ha17qf{background:#fff;border:1px solid rgb(218,220,224);border-radius:8px;display:block;-webkit-flex-shrink:0;flex-shrink:0;margin:0 auto;min-height:0;width:450px}.Ha17qf.qmmlRd{width:450px}.Ha17qf.qmmlRd .Or16q{height:auto;min-height:500px}}.Or16q{-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1;overflow:hidden;padding:24px 24px 36px}@media (min-width:601px){.Or16q{height:auto;min-height:500px;overflow-y:auto}}@media (min-width:450px){.Or16q{padding:48px 40px 36px}}.iEhbme{padding:24px 0 0}.iEhbme.RDPZE{opacity:.5;pointer-events:none}.BrpTO{margin:auto;max-width:380px;overflow:hidden;position:relative}.BrpTO .Q8ElWe{position:relative;text-align:center}.viAgtf{border-radius:50%;color:rgb(95,99,104);overflow:hidden}.eCirAf{line-height:1.4286}.cABCAe{width:100%}.cABCAe .viAgtf{-webkit-box-flex:0;-webkit-flex:none;flex:none;height:28px;margin-right:12px;width:28px}.cABCAe .Q8ElWe,.TPmpLe .Q8ElWe{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-align:center;-webkit-align-items:center;align-items:center}.cABCAe .Q8ElWe{-webkit-box-pack:center;-webkit-justify-content:center;justify-content:center}.BrpTO .viAgtf{height:64px;margin:0 auto 8px;width:64px}.rs3gSb{border-radius:50%;display:block}.cABCAe .rs3gSb,.cABCAe .hZUije,.cABCAe .kHluYc{max-height:100%;max-width:100%}.BrpTO .rs3gSb,.BrpTO .hZUije,.BrpTO .kHluYc{height:64px;width:64px}.TPmpLe{height:20px}.TPmpLe .viAgtf{display:-webkit-box;display:-webkit-flex;display:flex;height:20px;margin-right:8px;min-width:20px}.TPmpLe .rs3gSb,.TPmpLe .hZUije,.TPmpLe .kHluYc{color:rgb(60,64,67);height:20px;width:20px}.TPmpLe .kk39Eb{overflow:hidden}.TPmpLe .yavlK{overflow:hidden;text-overflow:ellipsis;white-space:nowrap}.cABCAe .kk39Eb{-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1}.cABCAe .eCirAf{color:rgb(60,64,67);font-size:14px;font-weight:500}.BrpTO .eCirAf{color:rgb(32,33,36);font-size:16px}.yavlK,.LAkAYd,.FzDwd{direction:ltr;font-size:12px;text-align:left;line-height:1.3333;word-break:break-all}.FzDwd{color:rgb(95,99,104)}.TPmpLe .yavlK{color:rgb(60,64,67)}.cABCAe .yavlK{color:rgb(95,99,104)}.cABCAe .LAkAYd{color:rgb(95,99,104)}.BrpTO .yavlK{color:rgb(95,99,104);text-align:center}.BtUzhd{color:rgb(95,99,104);font-size:12px}.cABCAe .BtUzhd{-webkit-align-self:flex-start;align-self:flex-start;-webkit-box-flex:0;-webkit-flex:none;flex:none;line-height:1.3333}.HDuqac{background:transparent;border:none;color:rgb(60,64,67);cursor:pointer;display:-webkit-inline-box;display:-webkit-inline-flex;display:inline-flex;font-size:14px;letter-spacing:.25px;max-width:100%}.BOs5fd{-webkit-box-align:center;-webkit-align-items:center;align-items:center;background:#fff;border:1px solid rgb(218,220,224);display:-webkit-inline-box;display:-webkit-inline-flex;display:inline-flex;max-width:100%;position:relative}.HDuqac:focus-visible{outline:none}.HDuqac:focus-visible .BOs5fd{outline:none;position:relative;background:rgba(60,64,67,.12)}.HDuqac:focus-visible .BOs5fd::after{border:2px solid rgb(24,90,188);border-radius:20px;bottom:-5px;box-shadow:0 0 0 2px rgb(232,240,254);content:"";left:-5px;position:absolute;right:-5px;top:-5px}.HDuqac:focus:not(:focus-visible) .BOs5fd{box-shadow:0 1px 1px 0 rgba(66,133,244,.3),0 1px 3px 1px rgba(66,133,244,.15);border-color:rgb(218,220,224);box-shadow:none}.HDuqac:hover:not(:focus-visible) .BOs5fd{background:rgba(60,64,67,.04)}.HDuqac:focus .BOs5fd,.HDuqac:hover .BOs5fd{border-color:rgb(218,220,224)}.HDuqac:active:focus .BOs5fd{background:rgba(60,64,67,.12);border-color:rgb(60,64,67);color:rgb(60,64,67)}.EI77qf{line-height:30px;margin:-8px 0;padding:8px 0}.EI77qf.DbQnIe{color:rgb(26,115,232);font-size:12px;line-height:22px}.EI77qf .BOs5fd{border-radius:16px;padding:0 15px 0 15px}.EI77qf.DbQnIe .BOs5fd{border-radius:12px;padding:0 10px 0 10px}.EI77qf.iiFyne .BOs5fd{padding-right:7px}.EI77qf.cd29Sd .BOs5fd{padding-left:5px}.EI77qf.DbQnIe.cd29Sd .BOs5fd{padding-left:2px}.EI77qf.DbQnIe.iiFyne .BOs5fd{padding-right:7px}.hMeYtd{border-radius:10px;height:20px;margin-right:8px}.hMeYtd .rs3gSb,.hMeYtd .hZUije,.hMeYtd .kHluYc{border-radius:50%;color:rgb(60,64,67);display:block;height:20px;width:20px}.wJxLsd{direction:ltr;overflow:hidden;text-align:left;text-overflow:ellipsis;white-space:nowrap}.znpTjf{color:rgb(60,64,67);-webkit-flex-shrink:0;flex-shrink:0;height:18px;margin-left:4px;width:18px}.HDuqac.DbQnIe .znpTjf{height:16px;width:16px}.JC0zZc{display:block;height:100%;width:100%}.aMfydd{text-align:center}.aMfydd .Tn0LBd{padding-bottom:0;padding-top:16px;color:rgb(32,33,36);font-size:24px;font-weight:400;line-height:1.3333;margin-bottom:0;margin-top:0}.a2CQh{padding-bottom:3px;padding-top:1px;margin-bottom:0;margin-top:0}.a2CQh{font-size:16px;font-weight:400;letter-spacing:.1px;line-height:1.5;padding-bottom:0;padding-top:8px}.a2CQh:empty{display:none}.n868rf{display:-webkit-inline-box;display:-webkit-inline-flex;display:inline-flex;letter-spacing:.25px;min-height:24px;padding-bottom:0;padding-top:8px}.C7uRJc{margin-top:8px}.NveWz{-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1}.i2knIc{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:reverse;-webkit-flex-direction:column-reverse;flex-direction:column-reverse;-webkit-box-flex:0;-webkit-flex-grow:0;flex-grow:0;-webkit-flex-wrap:wrap;flex-wrap:wrap;margin-left:-8px;margin-top:32px;margin-bottom:-16px;min-height:48px;padding-bottom:20px}.i2knIc.fXx9Lc,.i2knIc.fXx9Lc .RhTxBf,.i2knIc.fXx9Lc .tmMcIf{margin:0;min-height:0;padding:0}.sXlxWd{margin-bottom:32px;width:100%}.wg0fFb{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-orient:horizontal;-webkit-box-direction:reverse;-webkit-flex-direction:row-reverse;flex-direction:row-reverse;-webkit-flex-wrap:wrap;flex-wrap:wrap;width:100%}.RhTxBf,.tmMcIf{-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1;margin-bottom:16px}.i2knIc.NNItQ:not(.F8PBrb) .RhTxBf,.i2knIc.NNItQ:not(.F8PBrb) .tmMcIf{text-align:center}.RhTxBf{text-align:right}.i2knIc.F8PBrb{margin-left:0}.i2knIc.F8PBrb .wg0fFb{margin:0 -2px;width:calc(100% + 4px)}.i2knIc.F8PBrb .RhTxBf{display:-webkit-box;display:-webkit-flex;display:flex;-webkit-flex-wrap:wrap;flex-wrap:wrap;width:100%}.i2knIc.F8PBrb .tmMcIf{margin:0 2px}.xOs3Jc{-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1;margin:0 2px;min-width:calc(50% - 4px)}.Cokknd{margin-bottom:6px;margin-top:6px;white-space:nowrap;width:100%}.D4rY0b{color:rgb(95,99,104);font-size:14px;line-height:1.4286;margin-top:32px}.sQecwc{display:hidden}.TRuRhd{margin-top:24px;position:relative}.Fu5aXd:first-child .TRuRhd{margin-top:8px}.xyezD{background-color:transparent;border:none;box-sizing:border-box;color:rgb(32,33,36);font-size:16px;height:56px;outline:none;padding:0 14px;width:100%}.TRuRhd.YKooDc .xyezD{direction:ltr;text-align:left}.Yr2OOb{line-height:1.4286;margin:14px;padding:0;resize:vertical}.LBj8vb{background-color:transparent;border:none;font-size:16px;height:56px;padding:0 14px;outline:none;width:100%}.dXXNOd{display:-webkit-box;display:-webkit-flex;display:flex;overflow:hidden;text-overflow:ellipsis;white-space:nowrap;width:100%}.xMpNCd:not(:empty){line-height:56px;padding-left:14px}.pkBWge:not(:empty){line-height:56px;padding-right:14px}.TRuRhd[data-has-domain-suffix] .pkBWge{display:-webkit-box;display:-webkit-flex;display:flex;white-space:nowrap}.TRuRhd[data-has-domain-suffix][data-has-at-sign] .pkBWge{display:none}.fjpXlc{display:-webkit-box;display:-webkit-flex;display:flex;height:100%}.nWPx2e{-webkit-box-align:stretch;-webkit-align-items:stretch;align-items:stretch;display:-webkit-box;display:-webkit-flex;display:flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-webkit-flex-direction:row;flex-direction:row;height:100%;-webkit-box-pack:start;-webkit-justify-content:flex-start;justify-content:flex-start;left:0;max-width:100%;pointer-events:none;position:absolute;top:0;width:100%}.YhhY8,.CCQ94b,.tNASEf{border:1px solid rgb(218,220,224)}.YhhY8{border-bottom-left-radius:4px;border-right:none;border-top-left-radius:4px;width:8px}.CCQ94b{border-left:none;border-right:none;border-top:none;color:rgb(95,99,104);font-size:12px;margin:-6px 0 0;overflow:hidden;padding:0 6px;text-overflow:ellipsis;white-space:nowrap}.tNASEf{border-bottom-right-radius:4px;border-left:none;border-top-right-radius:4px;-webkit-box-flex:1;-webkit-flex-grow:1;flex-grow:1}.Fu5aXd:not(.Jj6Lae) .xyezD:focus+.nWPx2e .CCQ94b,.Fu5aXd:not(.Jj6Lae) .LBj8vb:focus+.nWPx2e .CCQ94b{color:rgb(26,115,232)}.xyezD:focus+.nWPx2e .YhhY8,.xyezD:focus+.nWPx2e .CCQ94b,.xyezD:focus+.nWPx2e .tNASEf,.LBj8vb:focus+.nWPx2e .YhhY8,.LBj8vb:focus+.nWPx2e .CCQ94b,.LBj8vb:focus+.nWPx2e .tNASEf{border-color:rgb(26,115,232);border-width:2px}.TMZ8p{-webkit-box-align:center;-webkit-align-items:center;align-items:center;box-sizing:content-box;cursor:pointer;display:-webkit-inline-box;display:-webkit-inline-flex;display:inline-flex;-webkit-box-flex:0;-webkit-flex:0 0 48px;flex:0 0 48px;height:48px;-webkit-box-pack:center;-webkit-justify-content:center;justify-content:center;line-height:0;position:relative;vertical-align:bottom;white-space:nowrap;width:48px}.TMZ8p .zSAiZc{height:48px;left:0;margin:0;opacity:0;outline:0;padding:0;position:absolute;top:0;width:48px}.RiAcXe{-webkit-box-align:center;-webkit-align-items:center;align-items:center;border:2px solid currentColor;border-radius:2px;box-sizing:border-box;display:-webkit-inline-box;display:-webkit-inline-flex;display:inline-flex;height:18px;-webkit-box-pack:center;-webkit-justify-content:center;justify-content:center;left:15px;pointer-events:none;position:absolute;top:15px;width:18px}.TIX6ke{bottom:0;left:0;opacity:0;position:absolute;right:0;top:0;width:100%}.TMZ8p .zSAiZc~.RiAcXe .TIX6ke{color:white;opacity:1}.M08VSe{stroke:currentColor;stroke-dashoffset:29.7833385;stroke-width:3.12px}.TMZ8p .zSAiZc:enabled:not(:checked)~.RiAcXe{background-color:transparent;border-color:rgb(95,99,104)}.TMZ8p .zSAiZc:enabled:checked~.RiAcXe{background-color:rgb(26,115,232);border-color:rgb(26,115,232)}.TMZ8p .zSAiZc[disabled]:not(:checked)~.RiAcXe{background-color:transparent;border-color:rgba(60,64,67,.38)}.TMZ8p .zSAiZc[disabled]:checked~.RiAcXe{background-color:rgba(60,64,67,.38);border-color:transparent}.TMZ8p:hover .zSAiZc:enabled:not(:checked)~.RiAcXe{background-color:transparent;border-color:rgb(32,33,36)}.TMZ8p:hover .zSAiZc:enabled:checked~.RiAcXe{background-color:rgb(23,78,166);border-color:rgb(23,78,166)}.TMZ8p .zSAiZc:focus-visible~.RiAcXe{outline:none;position:relative}.TMZ8p .zSAiZc:focus-visible~.RiAcXe::after{border:2px solid rgb(24,90,188);border-radius:4px;bottom:-5px;box-shadow:0 0 0 2px rgb(232,240,254);content:"";left:-5px;position:absolute;right:-5px;top:-5px}.F3wxlc{-webkit-box-align:start;-webkit-align-items:flex-start;align-items:flex-start;color:rgb(95,99,104);display:-webkit-box;display:-webkit-flex;display:flex;font-size:12px;line-height:normal;margin-top:4px}.EllNBf{margin-right:8px;margin-top:-2px}.SnjiRb{height:16px;width:16px}.F3wxlc:empty,.NHVGlc:empty{display:none}.Fu5aXd.Jj6Lae .F3wxlc{color:rgb(217,48,37)}.Fu5aXd .azsAwf{margin-left:16px}.Fu5aXd.Jj6Lae .nWPx2e .YhhY8,.Fu5aXd.Jj6Lae .nWPx2e .CCQ94b,.Fu5aXd.Jj6Lae .nWPx2e .tNASEf{border-color:rgb(217,48,37)}.Fu5aXd.Jj6Lae .nWPx2e .CCQ94b{color:rgb(217,48,37)}.ZWssT{margin-top:26px}.vopC4e{background:transparent;border:none;box-sizing:border-box;color:rgb(32,33,36);cursor:pointer;margin-bottom:-15px;margin-top:-15px;outline:inherit;padding-bottom:15px;padding-top:15px;position:relative;z-index:1}.vopC4e:focus::after{background-color:rgba(26,115,232,.15);border-radius:2px;bottom:0;content:"";left:0;position:absolute;right:0;top:0;z-index:-1}.JVMrYb{display:block}.hJIRO{display:none}sentinel{}
  </style>
  <style nonce="TaSeLqc4uqf3ONIHP6MfHw">
   @font-face{font-family:'Product Sans';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/productsans/v9/pxiDypQkot1TnFhsFMOfGShVF9eL.ttf)format('truetype');}
  </style>
  <meta content="nointentdetection" name="chrome"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="" name="description"/>
  <noscript>
   <meta content="0; url=/v3/signin/rejected?continue=https://accounts.google.com/&amp;dsh=S2047708041:1722537271087975&amp;flowEntry=ServiceLogin&amp;flowName=WebLiteSignIn&amp;followup=https://accounts.google.com/&amp;ifkv=AdF4I75IO5oamAg1uIOr74ZV4C80M5Js5Ko4E2YrpOZBLmdl_kkjf4y30Pf8PDRDVxu8dXHg5Q72&amp;rhlk=js&amp;rrk=47" http-equiv="refresh"/>
   <style nonce="TaSeLqc4uqf3ONIHP6MfHw">
    body{opacity:0;}
   </style>
  </noscript>
  <title>
   Sign in - Google Accounts
  </title>
 </head>
 <body>
  <div class="BDEI9 LZgQXe">
   <div class="Ha17qf" data-auto-init="Card">
    <div class="Or16q">
     <div data-allow-sign-up-types="true" data-locale="en_US" data-view-id="hm18Ec">
      <c-wiz c-wiz="" data-node-index="2;0" data-p="%.@.]" jsdata="deferred-i2" jsmodel="hc6Ubd" jsrenderer="OTcFib" jsshadow="">
       <div class="gEc4r">
        <img alt="Google" class="TrZEUc" height="24" src="//ssl.gstatic.com/images/branding/googlelogo/2x/googlelogo_color_74x24dp.png" width="74"/>
       </div>
       <c-data id="i2" jsdata=" eCjdDd;_;2">
       </c-data>
      </c-wiz>
      <div class="EQIoSc" jsname="bN97Pc">
       <div jsname="paFcre">
        <div class="aMfydd" jsname="tJHJj">
         <h1 class="Tn0LBd" jsname="r4nke">
          Sign in
         </h1>
         <p class="a2CQh" jsname="VdSJob">
          Use your Google Account
         </p>
        </div>
       </div>
       <form action="/v3/signin/identifier?continue=https://accounts.google.com/&amp;dsh=S2047708041:1722537271087975&amp;flowEntry=ServiceLogin&amp;flowName=WebLiteSignIn&amp;followup=https://accounts.google.com/&amp;ifkv=AdF4I75IO5oamAg1uIOr74ZV4C80M5Js5Ko4E2YrpOZBLmdl_kkjf4y30Pf8PDRDVxu8dXHg5Q72" method="POST" novalidate="">
        <div class="iEhbme" jsname="rEuO1b">
         <section class="aN1Vld">
          <div class="yOnVIb" jsname="MZArnb">
           <div class="Fu5aXd" jsname="dWPKW">
            <div class="Flfooc">
             <div class="TRuRhd YKooDc">
              <div class="fjpXlc">
               <label class="dXXNOd">
                <input autocapitalize="none" autocomplete="username" autofocus="" class="xyezD" dir="ltr" id="identifierId" jsname="Ufn6O" name="identifier" type="email"/>
                <div class="nWPx2e">
                 <div class="YhhY8">
                 </div>
                 <div class="CCQ94b">
                  Email or phone
                 </div>
                 <div class="tNASEf">
                 </div>
                </div>
               </label>
              </div>
             </div>
            </div>
            <div class="F3wxlc" jsname="h9d3hd">
            </div>
            <div class="NHVGlc" jsname="JIbuQc">
            </div>
           </div>
           <p aria-live="assertive" class="vOZun" jsname="OZNMeb">
           </p>
           <p class="vOZun">
            <a href="/signin/usernamerecovery?continue=https://accounts.google.com/&amp;dsh=S2047708041:1722537271087975&amp;flowEntry=ServiceLogin&amp;flowName=WebLiteSignIn&amp;followup=https://accounts.google.com/&amp;ifkv=AdF4I75IO5oamAg1uIOr74ZV4C80M5Js5Ko4E2YrpOZBLmdl_kkjf4y30Pf8PDRDVxu8dXHg5Q72" jsname="Cuz2Ue">
             Forgot email?
            </a>
           </p>
           <input aria-hidden="true" class="hJIRO" jsname="RHeR4d" name="hiddenPassword" spellcheck="false" tabindex="-1" type="password"/>
           <input name="usi" type="hidden" value="S2047708041:1722537271087975"/>
           <input name="domain" type="hidden" value=""/>
           <input name="region" type="hidden" value="SG"/>
           <span jsname="xdJtEf">"""

In [ ]:
# text = soup.get_text(separator='\n', strip=True)
model_classifier(text, model, tokenizer)

In [22]:

!nvidia-smi


Thu Aug  1 18:35:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0              28W /  70W |   7829MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
import gc
gc.collect()

1152